# Trained Yolo Model Code

### 1. Install Ultralytics Library

In [ ]:
!pip install ultralytics

### 2. Import library and make instance of yolo model

In [ ]:
from ultralytics import YOLO

model_v10 = YOLO('yolov10n.yaml')

### 3. Mounting Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 4. Change your current directory location from content to keep your results save

In [ ]:
%cd /content/drive/MyDrive

### 5. Training Model script

In [ ]:
train_v10 = model_v10.train(data="/content/drive/MyDrive/morning_anarkali_wdataset/data.yaml", ##give path of your yoolo dataset format yaml file
                            epochs = 150,
                            batch = 16,
                            imgsz = 640,
                            save=True)

### 6. Validate the Custom Trained Model

In [ ]:
custom_v10 = YOLO("/content/drive/MyDrive/yolov10/runs/detect/train/weights/best.pt")
metrics_v10 = custom_v10.val(data="/content/drive/MyDrive/morning_anarkali_wdataset/data.yaml", split='test', imgsz=640)
print("--------------------------------------------------------")
print("Precision:- ", metrics_v10.box.p)
print("Recall:- ", metrics_v10.box.r)
print("F1-Score:- ", metrics_v10.box.f1)
print("map@50:- ", metrics_v10.box.map50)
print("map@0.5-0.95:- ", metrics_v10.box.map)
print("----------------------------------------------------------")

# Tracking and Counting Code

In [ ]:
%cd /content

### 7. Cloning DeepSort Repository

In [ ]:
!git clone https://github.com/nwojke/deep_sort.git

### 8. Install Dependencies and Test Import Repository

In [ ]:
%cd deep_sort
!pip install -r requirements.txt

In [ ]:
import sys
sys.path.append("/content/deep_sort")

from deep_sort.deep_sort import DeepSort

### 9. Tracking and Counting Logic

In [ ]:
## importing required libraries
import cv2
import torch
import numpy as np
import csv
import sys
from ultralytics import YOLO
from torchvision.ops import nms

# DeepSORT setup
sys.path.append('/content/drive/MyDrive/') ## give path of directory here where deep_sort (cloned repository folder) is located
from deep_sort.deep_sort import DeepSort

# Load YOLO model
yolo_model = YOLO('/content/drive/MyDrive/best.pt') ## give path of your custom trained model weights
yolo_model.conf = 0.5

# DeepSORT instance
deepsort = DeepSort(
    model_path='/content/drive/MyDrive/deep_sort/deep/checkpoint/ckpt.t7', ## deep_sort tracker weights path
    max_dist=0.2,
    max_iou_distance=0.9,
    max_age=50,
    n_init=2,
    nn_budget=100,
    use_cuda=torch.cuda.is_available()
)

# Video I/O
cap = cv2.VideoCapture('/content/A8.mp4') ### your input video path on which your model is trained
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('/content/drive/MyDrive/A8_track.mp4', fourcc, ## output processed video path
                      int(cap.get(cv2.CAP_PROP_FPS)),
                      (int(cap.get(3)), int(cap.get(4))))

# CSV Logging setup
csv_path = "/content/drive/MyDrive/A8_track.csv" ## output processed video csv file path
csv_file = open(csv_path, mode='w', newline='')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['frame', 'track_id', 'x1', 'y1', 'x2', 'y2'])

# Tracking setup
unique_ids = set()
track_frame_counts = {}
frame_number = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_number += 1
    result = yolo_model(frame)[0]

    xywhs, confs, boxes_nms = [], [], []

    for box, cls_id, conf in zip(result.boxes.xyxy.cpu().numpy(),
                                 result.boxes.cls.cpu().numpy(),
                                 result.boxes.conf.cpu().numpy()):
        if int(cls_id) != 0:
            continue
        x1, y1, x2, y2 = box.astype(int)
        w, h = x2 - x1, y2 - y1
        cx, cy = x1 + w // 2, y1 + h // 2
        xywhs.append([cx, cy, w, h])
        confs.append(conf)
        boxes_nms.append([x1, y1, x2, y2])

    # Apply NMS
    if len(boxes_nms) > 1:
        boxes_tensor = torch.tensor(boxes_nms, dtype=torch.float32)
        confs_tensor = torch.tensor(confs, dtype=torch.float32)
        keep_idxs = nms(boxes_tensor, confs_tensor, iou_threshold=0.5)
        xywhs = [xywhs[i] for i in keep_idxs]
        confs = [confs[i] for i in keep_idxs]

    if xywhs:
        xywhs = np.array(xywhs)
        confs = np.array(confs)
        outputs = deepsort.update(xywhs, confs, frame)
    else:
        outputs = []

    for x1, y1, x2, y2, track_id in outputs:
        track_frame_counts[track_id] = track_frame_counts.get(track_id, 0) + 1
        if track_frame_counts[track_id] >= 4:
            unique_ids.add(track_id)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (0,255,0), 2)
        cv2.putText(frame, f"waste-{track_id}", (x1, y1-10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

        # Write to CSV
        csv_writer.writerow([frame_number, track_id, x1, y1, x2, y2])

    cv2.putText(frame,
                f"Unique Waste Count: {len(unique_ids)}",
                (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0,255), 3)

    out.write(frame)

cap.release()
out.release()
csv_file.close()
cv2.destroyAllWindows()